In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [1]:
import pandas as pd
import os
import re
import codecs
from IPython.display import display
from six.moves import cPickle as pickle
import string
from PIL import Image
import numpy as np
import h5py

In [2]:
width = None
pd.options.display.max_rows = 600
pd.options.display.max_columns = width
pd.options.display.max_colwidth = 600
pd.options.display.width = width
pd.options.display.max_seq_items = None
pd.options.display.expand_frame_repr = False
pd.options.display.colheader_justify = 'left'

In [47]:
class Visualize(object):
    def __init__(self, storedir, gen_datadir):
        self._storedir = storedir
        self._word2id = pd.read_pickle(os.path.join(gen_datadir, 'dict_vocab.pkl'))
        i2w = pd.read_pickle(os.path.join(gen_datadir, 'dict_id2word.pkl'))
        for i in range(-1,-11,-1):
            i2w[i] = '%d'%i
        self._id2word = {}
        ## Append space after all commands beginning with a backslash (except backslash alone)
        for i, w in i2w.items():
            if w[0] == '\\':
              self._id2word[i] = w + " "  
            else:
                self._id2word[i] = w 
        self._id2word[self._word2id['id']['\\']] = '\\'
    
    @property
    def w2i(self):
        return self._word2id['id']

    @property
    def i2w(self):
        return self._id2word
    
    @property
    def max_step(self):
        steps = [int(os.path.basename(f).split('_')[-1].split('.')[0]) for f in os.listdir(self._storedir)]
        return sorted(steps)[-1]
        
    
    def keys(self, graph, step):
        with h5py.File(os.path.join(self._storedir, '%s_%d.h5'%(graph,step))) as h5:
            return h5.keys()

    def np(self, graph, step, key=None):
        """
        Args:
            graph: 'training' or 'validation'
            step:  step who's output is to be fetched
            key:   key of object to fetch - e.g. 'predicted_ids'
        """
        with h5py.File(os.path.join(self._storedir, '%s_%d.h5'%(graph,step))) as h5:
            return h5[key][...]
    
    def df(self, graph, step, key):
        return pd.DataFrame(self.np(graph, step, key))
    
    def words(self, graph, step, key):
        df = self.df(graph, step, key)
        return df.applymap(lambda x: self._id2word[x])

    def strs(self, graph, step, key):
        df_str = self.words(graph, step, key)
        
        ## each token's string version - excepting backslash - has a space appended to it,
        ## therefore the string output should be compile if the prediction was syntactically correct
        return pd.DataFrame(["".join(row) for row in df_str.itertuples(index=False)])

class VisualizeStep():
    def __init__(self, visualizer, step):
        self._step = step
        self._visualizer = visualizer
        
    def keys(self, graph):
        return self._visualizer.keys(graph, self._step)
    
    def np(self, graph, key=None):
        return self._visualizer.np(graph, self._step, key)
    
    def df(self, graph, step, key):
        return pd.DataFrame.df(self.np(graph, step, key))
    
    def words(self, graph, key):
        return self._visualizer.words(graph, self._step, key)

    def strs(self, graph, key):
        return self._visualizer.strs(graph, self._step, key)


In [62]:
# v = Visualize('./tb_metrics_dev/2017-10-06 17-56-47 PDT/store', '../data/generated2')
v = Visualize('./tb_metrics/2017-10-07 14-33-35 PDT/store', '../data/generated2')
v.max_step

6050

In [68]:
vs = VisualizeStep(v, 5180)
vs.keys('validation')

[u'alpha', u'predicted_ids', u'y']

In [69]:
vs.strs('validation', 'predicted_ids')

,0
0,F=\frac {1}{2}\left (\frac {1}{2}-\frac {1}{2})\eos \eos \eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1
1,S=ed^{2}\phi ^{2}\phi ^{2}(x^{2})^{2}\phi ^{2}^{2})\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1
2,\phi (x)=\frac {1}{2}\left (x^{2}x^{2}x^{2}(x_{1})^{2}x_{2}(x_{2}).\eos \eos \eos \eos \eos \eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1
3,"\label {eq:1}{\cal L}_{\mu }^{(1)}=0,\,\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\"
4,"x_{1}^{2}=\frac {2}{2}\,\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;"
5,"\label {eq:nerer}{\cal L}_{e}={\frac {1}{2\pi ^{2}^{2}}\frac {1}{2\pi ^{2}^{2}}\,\frac {1}{2\pi ^{2}}\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1"
6,"\label {eq:1}\frac {a}{r}\,\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\"
7,\label {eq:1}d^{2}_{1}=x_{2}^{2}+r_{2}^{2}+x_{2}^{2}\eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1
8,"F(x,x)=(\phi ^{(1)}^{(1)})^{(1)}_{(0)}^{(0)}^{(0)}_{(0)}^{(0)}_{(0)}^{(1)}_{(0)}^{(1)}_{(1)}^{(1)}_{(1)}^{(1)},\eos \eos \eos \eos \eos \eos \eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1"
9,\label {1.1}[A^{(1)}^{(1)}^{(1)}^{(1)}^{(1)}=0\eos \eos \eos -1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1-1


In [70]:
vs.strs('validation', 'y')

,0
0,K=\frac {1}{R}\ln \left (-\frac {V_1}{V_2}\right )\label {K1}\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
1,\label {chiral}S=\int d^2x(\dot \phi \phi '-\phi '\phi ')\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
2,"\varphi (x)\equiv \int \textrm {d}^4p\,\tilde \varphi (p)\exp (ip^\mu x_\mu ).\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
3,\partial _+\Omega |_{cr}=0=\partial _-\Omega |_{cr}\label {bcon}\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
4,"\hat {a}\rightarrow \sum _{a=1}^{m^{2}}\hat {a}^{a}\otimes T^{a},\eos \eos \eos \eos \eos \eos"
5,"g_{\alpha \bar \beta }={\partial \over \partial w^\alpha }{\partial \over \partial \bar w^\beta }K,\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
6,"i,j=1\ldots d\quad \sp \mu ,\nu =0\ldots (9-d)\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"
7,\omega _{S_2}=\lambda _+\omega _{F_+}+\lambda _-\omega _{F_-}\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos
8,"K(\phi ,\phi ^*)=\Big (\phi ^\dagger \phi \Big )+G_\lambda (\phi ,\phi ^*,0,0\cdots )\eos \eos \eos \eos \eos \eos \eos \eos"
9,"\left \langle \left <\,\mathbf {F}_{\mu \nu }(x)\,\right >\right \rangle =0\eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos \eos"


In [21]:
v.w2i['\\']

61